# CSCI 2470 Final Project

## Terms

Let's use `c` to denote constants (e.g., `1` and `2` and `π` and `e`) and `x` to denote variables/parameters (e.g., `x`). A term `t` is defined as

```
t ::= c
    | x
    | sin(t)
    | - t
    | tᵗ
    | t + ... + t
    | t - ... - t
    | t × ... × t
```

In [120]:
import math
from math import sin, pow

class Term:
    def __str__(self) -> str:
        raise NotImplementedError
    def __repr__(self) -> str:
        return str(self)
    def value(self, x):
        return NotImplementedError
    def size(self):
        return NotImplementedError

class Constant(Term):
    def __init__(self, c) -> None:
        super().__init__()
        self.c = c
    def __str__(self) -> str:
        return str(self.c)
    def value(self, x):
        return self.c
    def size(self):
        return 1

class Variable(Term):
    def __init__(self) -> None:
        super().__init__()
    def __str__(self) -> str:
        return "x"
    def value(self, x):
        return x
    def size(self):
        return 1

class UnaryOperation(Term):
    def __init__(self, t) -> None:
        super().__init__()
        self.t = t
    def f(self, v):
        return NotImplementedError
    def value(self, x):
        return self.f(self.t.value(x))
    def size(self):
        return self.t.size() + 1
class Sin(UnaryOperation):
    def __str__(self) -> str:
        return f"sin({str(self.t)})"
    def f(self, v):
        return sin(v)
class Neg(UnaryOperation):
    def __str__(self) -> str:
        return f"-{str(self.t)}"
    def f(self, v):
        return - v

class BinaryOperation(Term):
    def __init__(self, t1, t2) -> None:
        super().__init__()
        self.t1 = t1
        self.t2 = t2
    def f(self, v1, v2):
        return NotImplementedError
    def value(self, x):
        return self.f(self.t1.value(x), self.t2.value(x))
    def size(self):
        return self.t1.size() + self.t2.size() + 1
class Power(BinaryOperation):
    def __str__(self) -> str:
        return f"({str(self.t1)} ^ {str(self.t2)})"
    def f(self, v1, v2):
        return pow(v1, v2)

class VariadicOperation:
    def __init__(self, ts) -> None:
        super().__init__()
        assert isinstance(ts, list) and len(ts) >= 2, f"expecting a list of at least two arguments, give {ts}"
        self.ts = ts
    def op(self):
        return NotImplementedError
    def __str__(self) -> str:
        return f"({f' {self.op()} '.join(str(t) for t in self.ts)})"
    def f(self, vs):
        return NotImplementedError
    def value(self, x):
        return self.f([t.value(x) for t in self.ts])
    def size(self):
        return sum(t.size() for t in self.ts) + 1
class Add(VariadicOperation):
    def op(self):
        return "+"
    def f(self, vs):
        return sum(vs)
class Sub(VariadicOperation):
    def op(self):
        return "-"
    def f(self, vs):
        v = vs[0]
        for u in vs[1:]:
            v -= u
        return v
class Mul(VariadicOperation):
    def op(self):
        return "*"
    def f(self, vs):
        v = vs[0]
        for u in vs[1:]:
            v *= u
        return v

In practice, we ensure that the constants are reader-friendly.

In [121]:
import random

class Config:
    def __init__(self, width_limit, size_limit) -> None:
        assert width_limit >= 2
        self.width = lambda: random.randint(2, width_limit + 1)
        self.size_limit = size_limit

def make_term(config, depth):
    term0_makers = [
        lambda depth: Constant(random.choice([0, 1, 2, math.pi, math.e])),
        lambda depth: Variable()
    ]
    term1_makers = [
        lambda depth: Sin(make(depth)),
        lambda depth: Neg(make(depth)),
    ]
    term2_makers = [
        lambda depth: Power(make(depth), make(depth)),
    ]
    termN_makers = [
        lambda depth: Add(makeN(depth)),
        lambda depth: Sub(makeN(depth)),
        lambda depth: Mul(makeN(depth)),
    ]
    def makeN(depth):
        w = config.width()
        # print(f"width = {w}; depth = {depth}" )
        ts = random.sample(
            [
                *term0_makers,
                *term1_makers,
                *term2_makers,
                *termN_makers,
            ],
            w
        )
        return [ t(depth) for t in ts ]
    def make(depth):
        if depth == 0:
            return random.choice(term0_makers)(0)
        else:
            return random.choice([
                *term0_makers,
                *term1_makers,
                *term2_makers,
                *termN_makers,
            ])(depth - 1)
    while True:
        t = make(depth)
        if t.size() < config.size_limit:
            return t

In [170]:
t = make_term(Config(2, 20), 2)
str(t)

'(-((((x + 0) + 1) * -x) - (x ^ 1)) * ((x ^ x) ^ x))'